In [1]:
wd <- dirname(dirname(getwd()))
source(paste0(wd,"/mission_control/treasure_map.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/eval_help.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/paper_settings.R"))
source(paste0(wd,"/mission_control/helpers/xgboost/shapley_help.R"))

library(tidyverse)
library(xgboost)
library(survival)
library(survcomp)

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.1
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘purrr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”Warning message:
“package ‘forcats’ was built under R version 3.6.3”── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### 0 - Read HMF prepped data

In [2]:
cpi <- (
    readRDS(paste0(TMP_DIR, "validation-hmf-go.Rds")) 
        %>% drop_na(os, bor)
        %>% mutate_at(vars("tmb", "tcell","tgfb","prolif","pretreat", "purity"), scale) 
)

In [3]:
model_features <- c("tmb", "tcell", "prolif", "tgfb", "pretreat", "purity")

### 1 - Make builder data structure for data storage

In [4]:
builder <- function( df ){
    files <- list()
    files[['X']] =  as.matrix( df %>% select(all_of(model_features)))
    files[['id']] = df %>% select(patient_id, age, gender, bor)
    files[['complete_id']] = df %>% drop_na(tcell) %>% pull(patient_id)
    files[['Y_lr']] = df %>% pull(bor)
    files[['Y_os']] = df %>% pull(os)
    files
}

In [5]:
store <- list()
store[['all']] <- builder( cpi )
for (i in unique(cpi %>% pull(tissue))) store[[i]] <- builder( cpi %>% filter(tissue == i) )

### 2 - Fit and Store Models
- First fit Pan-Cancer Model
- Then fit tissue specific models with Pan-Cancer as the base

In [6]:
store[['all']][['mod_lr']] <- grid_fit( store[['all']]$Y_lr, 
                                        store[['all']]$X, 
                                        parameter_grid, 
                                        model = "lr" )$best_model 
store[['all']][['mod_os']] <- grid_fit( store[['all']]$Y_os, 
                                        store[['all']]$X, 
                                        parameter_grid, 
                                        model = "os" )$best_model

for (i in unique(cpi %>% pull(tissue))) {
    print(i)
    flush.console()
    store[[i]][['mod_lr']] <- grid_fit( store[[i]]$Y_lr, 
                                        store[[i]]$X, 
                                        parameter_grid, 
                                        model = "lr", 
                                        base_model = store[['all']][['mod_lr']])$best_model 
    store[[i]][['mod_os']] <- grid_fit( store[[i]]$Y_os, 
                                        store[[i]]$X, 
                                        parameter_grid, 
                                        model = "os",
                                        base_model = store[['all']][['mod_os']]
                                      )$best_model 
}

[1] "lung"
[1] "skin"
[1] "other"
[1] "bladder"


### 3 - Store Shaps

In [7]:
shap_binder <- function( i, model = "lr" ){

    goods <- store[[i]]
    X <- goods$X
    
    if( model == "lr"){
        mod <- goods$mod_lr
    } else {
        mod <- goods$mod_os
    }    
    
    ### extract Shapleys
    shaps <- data.frame( predict( mod , X, predcontrib = TRUE) )
    shaps$pred <- predict( mod , X, predcontrib = FALSE) 
    shaps$model <- i
    shaps$patient_id <- goods$id$patient_id
    shaps$age <- goods$id$age
    shaps$gender <- goods$id$gender
    shaps$response <- goods$id$response

    shaps %>% select(patient_id, model, everything())
}

In [8]:
for (i in names(store)) {
    print(i)
    flush.console()
    store[[i]][['shaps_lr']] <- shap_binder(i,"lr")
    store[[i]][['shaps_os']] <- shap_binder(i,"os")
}

[1] "all"
[1] "lung"
[1] "skin"
[1] "other"
[1] "bladder"


### 4 - Format Data for Output plots

In [9]:
name_map <- list( "tmb" = "TMB", 
                  "tcell" = "T-cell", 
                  "prolif" = "Proliferation", 
                  "tgfb" = "TGFB", 
                  "pretreat" = "Prior Systemic Therapy",
                  "purity" = "Purity")

In [10]:
chunker <- function( dataset, model, feature){
    
    goods <- store[[dataset]]
    X <- goods$X
    if( model == "lr"){
        shaps <- goods$shaps_lr
    } else {
        shaps <- goods$shaps_os
    }
    
    chunk <- data.frame( scaled_feature = X[,feature], shap_feature = shaps[,feature])
    chunk$feature <- name_map[[feature]]
    chunk$model <- model
    chunk$dataset <- dataset
    chunk$patient_id <- shaps$patient_id
    chunk$age <- shaps$age
    chunk$gender <- shaps$gender
    chunk$pred <- shaps$pred
    chunk$response <- shaps$response

    chunk %>% select( model, dataset, feature, everything())
}

In [11]:
p1 <- data.frame()
for ( model in c("os", "lr") ) {
    print(model)
    for ( dataset in names(store) ) {
        for ( feature in model_features)  {
            p1 <- rbind(p1, chunker(dataset, model, feature))
        }
    }
}

[1] "os"
[1] "lr"


In [12]:
ready <- p1 %>% group_by( dataset ) %>% arrange(scaled_feature) %>% ungroup()
ready$dataset <- str_to_title(ready$dataset)
ready$dataset <- ifelse( ready$dataset == "All", "Pan-Cancer (base-model)", ready$dataset)
ready$dataset <- factor(
    ready$dataset,
    levels = c("Pan-Cancer (base-model)", "Skin", "Lung", "Bladder", "Other")
)
ready$feature <- factor(
    ready$feature,
    levels = c("T-cell", "TMB", "Prior Systemic Therapy", "TGFB", "Proliferation", "Purity")
)

In [13]:
print("worked")

[1] "worked"


### 5 - Send it!

In [14]:
saveRDS( ready, paste0(TMP_DIR, "validation-hmf-shaps.Rds") )